# contents

- Speech processing basics: Recording, reading, resampling, writing, playing, trimming, plotting, etc.
- Windowing of speech
- Analyzing voiced speech frame (magnitude spectrum)
- (Short time) Fourier Transform (Spectrogram visualization)

## Import all the required packages

In [1]:
import scipy.io.wavfile as wav
import scipy.signal as sig
import numpy as np
import matplotlib.pyplot as plt
import sounddevice as sd
from scipy.io import wavfile

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Basic operation for speech sound

In [2]:
fs = 44100  # Sample rate/sampling frequency
seconds = 4  # Duration of recording

In [8]:
# Record 3 seconds of your speech (i.e,, Hello. This is a sound example by XX)
data = sd.rec(int(seconds * fs), samplerate=fs, channels=1)
sd.wait()  # Wait until recording is finished

In [9]:
# Play the recording
sd.play(data, fs)
sd.wait()  # Wait until file is done playing